# Scenario Simulation

This notebook demonstrates:
- Simulating promotional campaigns
- Simulating price changes
- What-if analysis and impact forecasting
- Comparing scenarios


In [ ]:
# Import simulation tools and libraries
import sys
from pathlib import Path
sys.path.insert(0, str(Path().absolute().parent))

from src.data_loader import load_cpg_data
from src.tools.scenario_simulation import simulate_promotion, simulate_price_change
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('seaborn-v0_8')
print("✅ Libraries imported successfully!")


In [ ]:
# Load baseline data
data_path = '../data/cpg_sales_data.parquet'

try:
    df = load_cpg_data(data_path)
    print(f"✅ Data loaded: {df.shape[0]:,} rows")
    
    # Show available products and stores for simulation
    if 'sku_id' in df.columns:
        print(f"\n📦 Available SKUs: {df['sku_id'].nunique()}")
        sample_sku = df['sku_id'].iloc[0]
        print(f"   Example SKU: {sample_sku}")
    
    if 'store_id' in df.columns:
        print(f"\n🏪 Available Stores: {df['store_id'].nunique()}")
        sample_store = df['store_id'].iloc[0]
        print(f"   Example Store: {sample_store}")
        
except FileNotFoundError:
    print(f"❌ Data file not found at {data_path}")
    df = None


## Promotion Simulation


In [ ]:
# Simulate promotion scenario
if df is not None:
    print("🎯 Simulating promotion scenario...")
    
    # Get sample product and store
    sample_sku = df['sku_id'].iloc[0] if 'sku_id' in df.columns else None
    sample_store = df['store_id'].iloc[0] if 'store_id' in df.columns else None
    
    promo_results = simulate_promotion(
        df,
        product_id=sample_sku,
        store_id=sample_store,
        discount_pct=0.15,  # 15% discount
        duration_days=7
    )
    
    print(f"✅ Promotion simulation complete!")
    print(f"\n📊 Promotion Impact:")
    print(f"   Baseline revenue: ${promo_results.get('baseline_revenue', 0):,.2f}")
    print(f"   Simulated revenue: ${promo_results.get('simulated_revenue', 0):,.2f}")
    print(f"   Revenue change: ${promo_results.get('revenue_change', 0):,.2f} "
          f"({promo_results.get('revenue_change_pct', 0):.2f}%)")
    print(f"   Units sold change: {promo_results.get('units_change', 0):.0f} "
          f"({promo_results.get('units_change_pct', 0):.2f}%)")
    
    # Visualize promotion impact
    if 'simulation_data' in promo_results:
        sim_data = promo_results['simulation_data']
        fig, axes = plt.subplots(1, 2, figsize=(15, 5))
        
        # Revenue comparison
        axes[0].bar(['Baseline', 'With Promotion'], 
                   [promo_results.get('baseline_revenue', 0),
                    promo_results.get('simulated_revenue', 0)],
                   color=['blue', 'green'])
        axes[0].set_title('Revenue Comparison')
        axes[0].set_ylabel('Revenue ($)')
        
        # Units comparison
        if 'baseline_units' in promo_results:
            axes[1].bar(['Baseline', 'With Promotion'],
                       [promo_results.get('baseline_units', 0),
                        promo_results.get('simulated_units', 0)],
                       color=['blue', 'green'])
            axes[1].set_title('Units Sold Comparison')
            axes[1].set_ylabel('Units')
        
        plt.tight_layout()
        plt.show()


## Price Change Simulation


In [ ]:
# Simulate price hike scenario
if df is not None:
    print("💰 Simulating price change scenario...")
    
    # Get sample product and store
    sample_sku = df['sku_id'].iloc[0] if 'sku_id' in df.columns else None
    sample_store = df['store_id'].iloc[0] if 'store_id' in df.columns else None
    
    price_results = simulate_price_change(
        df,
        product_id=sample_sku,
        store_id=sample_store,
        price_increase_pct=0.10  # 10% price increase
    )
    
    print(f"✅ Price change simulation complete!")
    print(f"\n📊 Price Change Impact:")
    print(f"   Baseline revenue: ${price_results.get('baseline_revenue', 0):,.2f}")
    print(f"   Simulated revenue: ${price_results.get('simulated_revenue', 0):,.2f}")
    print(f"   Revenue change: ${price_results.get('revenue_change', 0):,.2f} "
          f"({price_results.get('revenue_change_pct', 0):.2f}%)")
    print(f"   Units sold change: {price_results.get('units_change', 0):.0f} "
          f"({price_results.get('units_change_pct', 0):.2f}%)")
    print(f"   Price elasticity: {price_results.get('price_elasticity', 'N/A')}")
    
    # Visualize price change impact
    fig, axes = plt.subplots(1, 2, figsize=(15, 5))
    
    # Revenue comparison
    axes[0].bar(['Baseline', 'With Price Increase'], 
               [price_results.get('baseline_revenue', 0),
                price_results.get('simulated_revenue', 0)],
               color=['blue', 'orange'])
    axes[0].set_title('Revenue Comparison')
    axes[0].set_ylabel('Revenue ($)')
    
    # Units comparison
    if 'baseline_units' in price_results:
        axes[1].bar(['Baseline', 'With Price Increase'],
                   [price_results.get('baseline_units', 0),
                    price_results.get('simulated_units', 0)],
                   color=['blue', 'orange'])
        axes[1].set_title('Units Sold Comparison')
        axes[1].set_ylabel('Units')
    
    plt.tight_layout()
    plt.show()


## Scenario Comparison


In [ ]:
# Compare all scenarios
if df is not None and 'promo_results' in locals() and 'price_results' in locals():
    print("📊 Scenario Comparison:")
    print("=" * 60)
    
    scenarios = {
        'Baseline': {
            'revenue': promo_results.get('baseline_revenue', 0),
            'units': promo_results.get('baseline_units', 0)
        },
        '15% Promotion': {
            'revenue': promo_results.get('simulated_revenue', 0),
            'units': promo_results.get('simulated_units', 0)
        },
        '10% Price Increase': {
            'revenue': price_results.get('simulated_revenue', 0),
            'units': price_results.get('simulated_units', 0)
        }
    }
    
    comparison_df = pd.DataFrame(scenarios).T
    print(comparison_df)
    
    # Visualize comparison
    fig, axes = plt.subplots(1, 2, figsize=(15, 5))
    
    comparison_df['revenue'].plot(kind='bar', ax=axes[0], color=['blue', 'green', 'orange'])
    axes[0].set_title('Revenue by Scenario')
    axes[0].set_ylabel('Revenue ($)')
    axes[0].tick_params(axis='x', rotation=45)
    
    comparison_df['units'].plot(kind='bar', ax=axes[1], color=['blue', 'green', 'orange'])
    axes[1].set_title('Units Sold by Scenario')
    axes[1].set_ylabel('Units')
    axes[1].tick_params(axis='x', rotation=45)
    
    plt.tight_layout()
    plt.show()
    
    print("\n💡 Recommendations:")
    if promo_results.get('revenue_change', 0) > price_results.get('revenue_change', 0):
        print("   → Promotion scenario shows better revenue impact")
    else:
        print("   → Price increase scenario shows better revenue impact")
